# Génération automatique d'une synthèse ICV au format PDF

Ce notebook lit un fichier Excel contenant les bulletins ICV, génère une synthèse selon un format PDF de référence, et produit un document PDF final structuré par ligne.

In [ ]:
import pandas as pd
from openai import OpenAI
from fpdf import FPDF
import os
from secret_like import OPENAI_API_KEY # Remplacer par un fichier secret.py avec la clé api dedans

# Clé API (à définir manuellement ou via variable d'environnement)
client = OpenAI(api_key=OPENAI_API_KEY)

# Chargement du fichier Excel
csv_path = "IcvReferenceList_novembre.csv" # Assurez-vous que le fichier est bien dans le répertoire
df = pd.read_csv(csv_path, delimiter=";", quotechar='"').head(6)

# Colonnes pertinentes
df = df[[
    "Lignes",
    "Date de début",
    "Date de fin",
    "Catégorie",
    "Texte de l’ICV gare et bord"
]].dropna(subset=["Lignes"])

df


,Lignes,Date de début,Date de fin,Catégorie,Texte de l’ICV gare et bord
0,R,25/11/2024 03:50:21,28/12/2024 03:00:00,Travaux,NaN
1,C,08/09/2024 03:50:00,07/11/2024 20:00:00,Travaux,"RER C : en soirée, des travaux impactent votre..."
2,N,03/04/2024 06:00:00,08/11/2024 15:59:52,Travaux,Ligne N : des travaux impactent votre ligne. R...
3,N,03/04/2024 06:00:00,08/11/2024 15:59:52,Travaux,Ligne N : des travaux impactent votre ligne. R...
4,U,03/04/2024 06:00:00,08/11/2024 16:00:48,Travaux,Ligne U : des travaux impactent votre ligne. R...
5,U,03/04/2024 06:00:00,08/11/2024 16:00:48,Travaux,Ligne U : des travaux impactent votre ligne. R...


In [31]:
# Génération du prompt d'entrée pour l'API OpenAI (format basé sur l'exemple PDF)

def generate_prompt(dataframe):
    lignes = dataframe['Lignes'].unique()
    prompt = """Tu es un assistant de la SNCF. À partir des données suivantes extraites de bulletins ICV, rédige une synthèse claire, concise et structurée à destination du directeur opérationnel de ligne. Utilise le format suivant comme exemple :

Ligne C :
• Vacances Noël : Offre de transport réduite [...] 
• Accidents de personne : [...] 

Ligne T11 :
• Mouvement social local : [...] 
• Incident de signalisation : [...] 

N'invente rien. Utilise uniquement les informations ci-dessous :\n\n"""
    for ligne in lignes:
        sous_df = dataframe[dataframe['Lignes'] == ligne]
        for _, row in sous_df.iterrows():
            prompt += f"Ligne {row['Lignes']} | {row['Catégorie']} | {row['Date de début']} - {row['Date de fin']} | {row["Texte de l’ICV gare et bord"]} \n"
    prompt += "\nRédige maintenant une synthèse au format structuré ligne par ligne comme dans l’exemple PDF.\n"
    return prompt

prompt_text = generate_prompt(df)
print(prompt_text[:1500])  # Aperçu


Tu es un assistant de la SNCF. À partir des données suivantes extraites de bulletins ICV, rédige une synthèse claire, concise et structurée à destination du directeur opérationnel de ligne. Utilise le format suivant comme exemple :

Ligne C :
• Vacances Noël : Offre de transport réduite [...] 
• Accidents de personne : [...] 

Ligne T11 :
• Mouvement social local : [...] 
• Incident de signalisation : [...] 

N'invente rien. Utilise uniquement les informations ci-dessous :

Ligne R | Travaux | 25/11/2024 03:50:21 - 28/12/2024 03:00:00 | nan 
Ligne C | Travaux | 08/09/2024 03:50:00 - 07/11/2024 20:00:00 | RER C : en soirée, des travaux impactent votre ligne. Retrouvez les informations sur l'appli Ile de France Mobilités, le site Transilien.com, SNCF Connect ou votre appli de mobilité. 
Ligne N | Travaux | 03/04/2024 06:00:00 - 08/11/2024 15:59:52 | Ligne N : des travaux impactent votre ligne. Retrouvez toutes les informations sur l'appli Ile de France Mobilités, Transilien.com ou votre 

In [32]:
# Appel à l’API OpenAI (GPT-4 recommandé)

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Tu es un assistant expert en rédaction opérationnelle pour la SNCF."},
        {"role": "user", "content": prompt_text}
    ],
    temperature=0.4,
    max_tokens=1200
)

synthese_text = response.choices[0].message.content
print(synthese_text)


Ligne R :
• Travaux : Les travaux sur la Ligne R ont débuté le 25/11/2024 à 03:50:21 et se termineront le 28/12/2024 à 03:00:00. Aucune information supplémentaire n'a été fournie.

Ligne C :
• Travaux : Des travaux ont impacté la Ligne C du 08/09/2024 à 03:50:00 au 07/11/2024 à 20:00:00. Les informations détaillées étaient disponibles sur l'appli Ile de France Mobilités, le site Transilien.com, SNCF Connect et les différentes applications de mobilité.

Ligne N :
• Travaux : La Ligne N a été affectée par des travaux du 03/04/2024 à 06:00:00 au 08/11/2024 à 15:59:52. Les informations détaillées étaient disponibles sur l'appli Ile de France Mobilités, le site Transilien.com et les différentes applications de mobilité.

Ligne U :
• Travaux : Des travaux ont impacté la Ligne U du 03/04/2024 à 06:00:00 au 08/11/2024 à 16:00:48. Les informations détaillées étaient disponibles sur l'appli Ile de France Mobilités, le site Transilien.com et les différentes applications de mobilité.


In [33]:
# Classe FPDF modifiée pour supporter l'Unicode
class UnicodePDF(FPDF):
    def __init__(self):
        super().__init__()
        # Activer le support Unicode
        self.add_page()
        self.set_auto_page_break(auto=True, margin=15)

    def header(self):
        # Pas d'en-tête personnalisé
        pass

    def footer(self):
        # Pas de pied de page personnalisé
        pass
        
    def normalize_text(self, text):
        # Remplacer les caractères problématiques par des équivalents ASCII
        replacements = {
            '\u2019': "'",  # Apostrophe courbe
            '\u2018': "'",  # Guillemet simple gauche
            '\u201c': '"',  # Guillemet double gauche
            '\u201d': '"',  # Guillemet double droit
            '\u2013': '-',  # Tiret moyen
            '\u2014': '--', # Tiret long
            '\u2026': '...' # Points de suspension
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
            
        return text

In [36]:
def enregistrer_pdf_unique(nom_fichier, contenu_texte):
    pdf = UnicodePDF()
    
    # Configuration de la police
    pdf.set_font('Arial', '', 12)
    
    # Titre principal
    pdf.cell(0, 10, "Synthèse des perturbations réseau SNCF", 0, 1, 'C')
    pdf.ln(5)
    
    # Contenu
    pdf.set_font('Arial', '', 12)
    
    # Normaliser le texte pour remplacer les caractères Unicode problématiques
    texte_contenu = pdf.normalize_text(contenu_texte)
        
    # Diviser le texte en lignes pour un meilleur formattage
    for ligne_contenu in texte_contenu.split('\n'):
        pdf.multi_cell(0, 10, ligne_contenu)

    pdf.output(nom_fichier)

In [38]:
# Alternative sans police personnalisée si DejaVu n'est pas disponible
def enregistrer_pdf_alternative(nom_fichier, contenu_texte):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', '', 12)
    
    # Titre principal
    pdf.cell(0, 10, "Synthèse des perturbations réseau SNCF", 0, 1, 'C')
    pdf.ln(5)
    
    # Remplacer tous les caractères spéciaux problématiques
    replacements = {
        '\u2022': '-',  # Puce remplacée par tiret
        '\u2019': "'",  # Apostrophe courbe
        '\u2018': "'",  # Guillemet simple gauche
        '\u201c': '"',  # Guillemet double gauche
        '\u201d': '"',  # Guillemet double droit
        '\u2013': '-',  # Tiret moyen
        '\u2014': '--', # Tiret long
        '\u2026': '...' # Points de suspension
    }
    
    for old, new in replacements.items():
        contenu_texte = contenu_texte.replace(old, new)
    
    # Diviser le texte en lignes pour un meilleur formattage
    for ligne_contenu in contenu_texte.split('\n'):
        pdf.multi_cell(0, 10, ligne_contenu)
        
    pdf.output(nom_fichier)

In [39]:
# Sauvegarder toutes les synthèses dans un seul PDF
try:
    enregistrer_pdf_alternative("Synthese_ICV_Tout_Reseau.pdf", synthese_text)
    print("PDF généré avec succès!")
except Exception as e:
    print(f"Erreur lors de la génération du PDF: {e}")

PDF généré avec succès!
